In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()
import gzip
tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
rider_cust_canc_raw_file='1582100338-1582100338-CaptainOrderStatus-customerCancelled-20200218-0-0.json.gz'

def parse_time(x):
    return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

with gzip.open(rider_cust_canc_raw_file) as f:
    rider_cust_canc_raw=pd.read_json(rider_cust_canc_raw_file,lines=True)

rider_cust_canc_raw.head()
rider_cust_canc_raw.eventProps.iloc[2]
rider_cust_canc_raw.profile.iloc[2]
rider_cust_canc_raw.deviceInfo.iloc[2]

,eventName,ts,eventProps,profile,deviceInfo
0,CaptainOrderStatus-customerCancelled,20200218000118,{'cancelReason': 'order cancelled before rider...,{'all_identities': ['5d5c0cc2a24c96105e1bc469'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ..."
1,CaptainOrderStatus-customerCancelled,20200218000124,{'cancelReason': 'order cancelled before rider...,{'all_identities': ['naufalmahammad045@gmail.c...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
2,CaptainOrderStatus-customerCancelled,20200218000154,{'cancelReason': 'order cancelled before rider...,{'all_identities': ['5dc8dbf4907d93538e41bf34'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ..."
3,CaptainOrderStatus-customerCancelled,20200218000204,"{'cancelReason': 'Captain denied duty', 'city'...",{'all_identities': ['5e2b0f65257046721c75b267'...,"{'osVersion': '7.1.1', 'sdkVersion': '30601', ..."
4,CaptainOrderStatus-customerCancelled,20200218000212,"{'cancelReason': 'Captain denied duty', 'city'...",{'all_identities': ['5cc996a94fd5d569a7137323'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."


{'cancelReason': 'order cancelled before rider accepted',
 'city': 'Bangalore',
 'pickupClusters': 'Mahadevpura',
 'dropClusters': 'Mahadevpura',
 'paymentType': 'cash',
 'orderType': 'app',
 'clientId': 'self',
 'showOtp': 'true',
 'CT Source': 'API',
 'lat': 12.996605,
 'lng': 77.69233,
 'identity': '5ddbaf27a80df9312a1969f3',
 'serviceType': '57370b61a6855d70057417d1',
 'orderId': '5e4adc142806995062b9f742',
 'timeBucket': '00:00-00:30',
 'orderDate': '2020-02-18',
 'otp': '9760'}

{'all_identities': ['5dc8dbf4907d93538e41bf34',
  'kumarkavana72@gmail.com',
  '5ddbaf27a80df9312a1969f3'],
 'identity': '5dc8dbf4907d93538e41bf34,5ddbaf27a80df9312a1969f3',
 'platform': 'Android',
 'phone': 919148800660,
 'name': 'GOVINDA H',
 'email': 'kumarkavana72@gmail.com',
 'push_token': 'fcm:dNTAXIXVEqs:APA91bHHfYb1Fm3QgzJJftQcZrx6JcVFmax4WhQ3eS2R21raVkgF1NG2bbvFkD4_czeMjnG_x7XEvAKj5UVCwpYzFJlbThyPZed4e3UBaR46X8JG1auq_m0BOUzfW8yTP4EERSEhRVty'}

{'osVersion': '8.1.0',
 'sdkVersion': '30601',
 'make': 'vivo',
 'model': 'Others',
 'appVersion': '3.10.5',
 'browser': 'MobileApp',
 'dpi': 320,
 'dimensions': {'width': 68, 'height': 135, 'unit': 'mm'}}

In [5]:
from datetime import datetime
def parse_rider_cust_canc_rendered(file_name):
    def get_property_value_for_col(df,source_column,target_column,property_key):
        def get_property(payload,property_key):
            try:
                identity_index=[i for i, s in enumerate(payload) if property_key in s][0]
            except:
                return None
            value=list(payload.values())[identity_index]
            return value
        df.loc[:,target_column]=df[source_column].progress_apply(lambda x: get_property(x,property_key))
        return df
    def parse_time(x):
        return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

    with gzip.open(rider_cust_canc_raw_file) as f:
        rider_cust_canc_raw=pd.read_json(rider_cust_canc_raw_file,lines=True)

    rider_cust_canc_raw['time']=rider_cust_canc_raw['ts'].progress_apply(parse_time)

    source_column='eventProps'
    target_column='cancel_reason'
    property_key='cancelReason'

    rider_cust_canc_raw=get_property_value_for_col(rider_cust_canc_raw,source_column,target_column,property_key)

    source_column='eventProps'
    target_column='order_id'
    property_key='orderId'

    rider_cust_canc_raw=get_property_value_for_col(rider_cust_canc_raw,source_column,target_column,property_key)
    source_column='profile'
    target_column='captain_id'
    property_key='identity'
    rider_cust_canc_raw=get_property_value_for_col(rider_cust_canc_raw,source_column,target_column,property_key)
    
    source_column='deviceInfo'
    target_column='app_version'
    property_key='appVersion'
    rider_cust_canc_raw=get_property_value_for_col(rider_cust_canc_raw,source_column,target_column,property_key)

    return rider_cust_canc_raw

rider_cust_canc=parse_rider_cust_canc_rendered(rider_cust_canc_raw_file)


rider_cust_canc.head(2)

100%|██████████| 193839/193839 [00:00<00:00, 266446.42it/s]


,eventName,ts,eventProps,profile,deviceInfo,time,cancel_reason,order_id,captain_id,app_version
0,CaptainOrderStatus-customerCancelled,20200218000118,{'cancelReason': 'order cancelled before rider...,{'all_identities': ['5d5c0cc2a24c96105e1bc469'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ...",2020-02-18 00:01:18,order cancelled before rider accepted,5e4adbe9865d9017b53de9d6,"5d5c0cc2a24c96105e1bc469,5d5c1bbca24c96105e1bdcff",3.10.5
1,CaptainOrderStatus-customerCancelled,20200218000124,{'cancelReason': 'order cancelled before rider...,{'all_identities': ['naufalmahammad045@gmail.c...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-18 00:01:24,order cancelled before rider accepted,5e4adbd49a773437e4b7a900,"5bc427ab34ef0f19f85220a8,5d7769509f130346e2cd06b9",3.10.5


In [8]:
rider_cust_canc[['eventName','time','order_id','captain_id','app_version','cancel_reason']].to_csv('rider_cust_canc_20200218.csv',index=False)